### Домашнее задание к вебинару 1

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

В качестве метрик для оценки ранжирования могут использоваться ранговые коэффициенты корреляции (Кенделла, Спирмена), которые учитывают порядок, а не сами значения.

Также могут использоваться метрики, основанные на каскадной модели поведения пользователя, которая исходит из предпосылки, что первые $k$ предложений могут устроить клиента и он покинет сайт: ERR (Expected Reciprocal Rank), PFound. В них учитываются вероятности того, что пользователь прекратит просмотр на конкретном элементе.

**2) Доделать все функции, где стоит комментарий "сделать дома"**  
* hit_rate_at_k. 
* money_precision_at_k. 
* recall_at_k
* money_recall_at_k  
* map_k  
* ndcg_at_k  

In [478]:
import numpy as np
import math

In [348]:
# Пример рекомендованных и купленных товаров
recommended_list = [143, 1134, 991, 27, 523, 1543, 3345, 533, 11, 43, 200, 173, 129, 568, 569] #id товаров
bought_list = [143, 523, 15, 43, 100, 129] # id купленных товаров


In [349]:
print(f'Всего рекомендованных товаров - {len(recommended_list)},  всего купленных товаров - {len(bought_list)} \
из них - {np.isin(recommended_list, bought_list).sum()} товара были в рекомендованных')

Всего рекомендованных товаров - 15,  всего купленных товаров - 6 из них - 4 товара были в рекомендованных


### Hit Rate

In [350]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

#### hit_rate_at_k

In [351]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    
    return hit_rate_at_k

In [352]:
hit_rate_at_k(recommended_list, bought_list, k=10)

1

### Precision

In [353]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

#### money_precision_at_k

In [354]:
# Добавим цены на рекомендованные товары
prices_for_recommended = [350, 40, 100, 240, 25, 500, 180, 110, 220, 55, 190, 300, 80, 140, 340]

In [355]:
len(recommended_list) == len(prices_for_recommended)

True

In [356]:
def money_precision_at_k(recommended_list, bought_list, prices_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_list = np.array(prices_list)[:k]
    
    # определяем метки купленных товаров среди рекомендованных
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    # рассчитаем revenue от рекомендованных релеватных товаров скалярным произведением
    recommended_relevant_rev = np.dot(flags, prices_list)
    
    precision_at_k = recommended_relevant_rev/sum(prices_list)
    
    return precision_at_k
    

In [357]:
money_precision_at_k(recommended_list, bought_list, prices_for_recommended, k=5)

0.4966887417218543

### Recall

In [358]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall



In [359]:
recall(recommended_list, bought_list)

0.6666666666666666

#### recall_at_k

In [360]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recall_at_k = sum(flags)/len(bought_list)
    
    return recall_at_k

In [361]:
recall_at_k(recommended_list, bought_list, k=7)

0.3333333333333333

#### money_recall_at_k

In [362]:
# Добавим цены на купленные товары
prices_for_bought = list(np.random.randint(20,500,len(bought_list)))

In [363]:
# Заменим значения релевантных рекомедованных товаров на уже известные цены этих рекомендованных товаров
for i in np.where(np.isin(bought_list, recommended_list))[0]:  #возвращаем индексы релеватных рекомендованных элементов
    prices_for_bought[i] = prices_for_recommended[i]

In [364]:
prices_for_bought

[350, 40, 331, 240, 137, 500]

In [365]:
prices_for_recommended

[350, 40, 100, 240, 25, 500, 180, 110, 220, 55, 190, 300, 80, 140, 340]

In [366]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k = 5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recommended_relevant_rev = np.dot(flags, prices_recommended)
    
    money_recall_at_k = recommended_relevant_rev/sum(prices_bought)
    
    return money_recall_at_k

In [367]:
money_recall_at_k(recommended_list, bought_list, prices_for_recommended, prices_for_bought, k=5)

0.2346683354192741

### AP@K

In [368]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  
    

In [369]:
ap_k(recommended_list, bought_list, k=5)

0.7

### MAP@K|

In [370]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]
                      ] # юзер 3

In [371]:
# Используя имеющиеся функции
def map_k(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    result = np.average([ap_k(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return result

In [372]:
map_k(recommended_list_3_users, bought_list_3_users)

0.3333333333333333

### NDCG@K

In [373]:
import math

In [429]:
def dcg_at_k(recommended_list, bought_list, k=5):
    
    flags = np.isin(recommended_list[:k], bought_list)
    
    norm_coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    return np.dot(flags, norm_coeffs) /k


def ndcg_at_k(recommended_list, bought_list, k=5):
    
    ideal_dcg = dcg_at_k(recommended_list, recommended_list[:k], k)
    return dcg_at_k(recommended_list, bought_list, k) / ideal_dcg 

In [430]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.5176090586800538

In [433]:
#  Общая функция для расчета

def ndcg_k(recommended_list, bought_list, k=5):

    flags = np.isin(recommended_list[:k], bought_list)
    
    #рассчитаем DCG    
    coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    dcg = np.dot(flags, coeffs)
    
    #Рассчитаем идеальный DCG
    ideal_dcg = sum([1/(i+1) if i<=2 else 1/math.log2(i+1) for i in range(k)])
    
    ndcg = dcg/ideal_dcg
    
    return ndcg

In [434]:
ndcg_k(recommended_list, bought_list)

0.5176090586800538

### Mean Reciprocal Rank ( MRR@k )

In [474]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    rec_relevant_indices = np.nonzero(np.isin(recommended_list[:k], bought_list))[0]
    if len(rec_relevant_indices) == 0:
        return 0
    rr = 1/(rec_relevant_indices[0]+1) #т к расчет идет с 1 (в отличии от индексов, которые с нуля)
    return rr

def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    mrr = np.average([reciprocal_rank(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return mrr

In [475]:
reciprocal_rank(recommended_list, bought_list)

1.0

In [476]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]
                      ] # юзер 3

In [477]:
mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users)

0.3333333333333333